In [10]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random

In [2]:
#theta: latent orientation variable
#z_l,z_r: latent eye specific variable
#x_l,x_r: evidence input
#alpha_l,alpha_r: contrast variable
#sampling algorithm: gibbs sampling
#prior_z: prior distribution of z_l,z), [p_00,p_01,p_10,p_11]

In [29]:
def sample_theta(x_l,x_r,z_l,z_r,alpha_l,alpha_r):
    sum_zeta = alpha_l*z_l*np.exp(1j*x_l)+alpha_r*z_r*np.exp(1j*x_r)
    mu = np.angle(sum_zeta) #average of the distribution
    dis = np.abs(sum_zeta) #disperson of the variable
    return random.vonmises(mu,dis)

def sample_z_l(theta,x_l,z_l,z_r,alpha_l,prior_z):
    p_l_0 = 1/2/np.pi*np.array([prior_z[z_r_i] for z_r_i in z_r.astype(int)])
    p_l_1 = 1/2/np.pi/np.i0(alpha_l)*np.exp(alpha_l*np.cos(theta-x_l))*np.array([prior_z[z_r_i+2] for z_r_i in z_r.astype(int)])
    return (np.sign(p_l_1/(p_l_1+p_l_0)-random.random(x_l.shape))+1)/2


def sample_z_r(theta,x_r,z_l,z_r,alpha_r,prior_z):
    p_r_0 = 1/2/np.pi*np.array([prior_z[2*z_l_i] for z_l_i in z_l.astype(int)])
    p_r_1 = 1/2/np.pi/np.i0(alpha_r)*np.exp(alpha_r*np.cos(theta-x_r))*np.array([prior_z[2*z_l_i+1] for z_l_i in z_l.astype(int)])
    return (np.sign(p_r_1/(p_r_1+p_r_0)-random.random(x_r.shape))+1)/2

In [36]:
def simulation(x_l,x_r,alpha_l,alpha_r,prior_z,no_steps):
    theta_list = []
    z_l_list = []
    z_r_list = []
    z_l = random.randint(0,2,x_l.shape)
    z_r = random.randint(0,2,x_l.shape)
    theta = (random.random(x_l.shape)-0.5)*2*np.pi
    for i in range(no_steps):
        theta = sample_theta(x_l,x_r,z_l,z_r,alpha_l,alpha_r)
        theta_a = theta.copy()
        theta_list.append(theta_a)
        z_l = sample_z_l(theta,x_l,z_l,z_r,alpha_l,prior_z)
        z_l_a = z_l.copy()
        z_l_list.append(z_l_a)
        z_r = sample_z_r(theta,x_r,z_l,z_r,alpha_r,prior_z)
        z_r_a = z_r.copy()
        z_r_list.append(z_r_a)
    return np.array(theta_list).reshape(-1),np.array(z_l).reshape(-1),np.array(z_r).reshape(-1)



In [35]:
np.ones(10).copy()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [46]:
x_l,x_r,alpha_l,alpha_r,prior_z,no_steps = np.pi/4*np.ones(10000),-np.pi/4*np.ones(10000),2,2,np.array([1/4,1/4,1/4,1/4]),10000
theta,z_l,z_r = simulation(x_l,x_r,alpha_l,alpha_r,prior_z,no_steps)
print(len(theta))

100000000


In [41]:
def pi_count_1(pi_l,pi_r):
    '''lr = 00,10,01,11'''
    pi_r_1 = pi_r.copy()
    pi_r_1 = pi_r_1*2
    pi_class = pi_r_1+pi_l
    return np.array([len(np.where(pi_class==0)[0])/len(pi_l),len(np.where(pi_class==1)[0])/len(pi_l),len(np.where(pi_class==2)[0])/len(pi_l),len(np.where(pi_class==3)[0])/len(pi_l)])

In [47]:
print(pi_count_1(z_l,z_r))

[0.2632 0.2609 0.2609 0.215 ]


In [49]:
print(np.i0(8**0.5)/np.i0(2)**2/(np.i0(8**0.5)/np.i0(2)**2+3))

0.21431191183029213
